In [6]:
# Dependencies
import numpy as np
import pandas as pd
import os          
import matplotlib.pyplot as plt
import cv2
import datetime
import time
import joblib
from skimage.feature import graycomatrix, graycoprops

# Load the KNN models
knn_model = joblib.load('knn_models.joblib')

# Preparation
filters = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
class_names = ['NYAMAN', 'TIDAK NYAMAN']

# Input
path = input('Masukan nama file: ')
file_format = input('Masukan file format: ')
responden = input('Masukan nama responden: ')
temperature = input ('Masukan nilai temperature ruangan: ')

# Put on Directory
address = 'testing/'+path+'.'+file_format

# Camera Input, if using webcam or integrated thermal camera change to VideoCapture(0)
cap=cv2.VideoCapture(address)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# The Program
while cap.isOpened():
    _,img=cap.read()
    
    # width & height
    width  = cap.get(3)  # float `width`
    height = cap.get(4)  # float `height`
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Start of Time
    start = time.time()

    # img = cv2.imread(img)
    cv2.imwrite('temp.jpg', img)
    
    #Image preprocessing
    def calc_glcm_all_agls(image, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
        fig = cv2.imread(image) 
        gray = cv2.cvtColor(fig, cv2.COLOR_BGR2GRAY)
        h, w = gray.shape
        ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
        fig = gray[ymin:ymax, xmin:xmax]
        glcm = graycomatrix(fig, 
                            distances=dists, 
                            angles=agls, 
                            levels=lvl,
                            symmetric=sym, 
                            normed=norm)
        feature = []
        glcm_props = [propery for name in props for propery in graycoprops(glcm, name)[0]]
        for item in glcm_props:
                feature.append(item)
        return feature

    # ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
    properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'energy']
    glcm_all_agls = []
    #for i in df.groupby(['name']):
    glcm_all_agls.append(calc_glcm_all_agls(image="temp.jpg", props=properties))
    columns = []
    angles = ['0', '45', '90','135']
    for name in properties :
        for ang in angles:
            columns.append(name + "_" + ang)
    x = pd.DataFrame(glcm_all_agls,columns=columns)
    
    # Prediction using Models
    pred = knn_model.predict(x)
    pred_proba = knn_model.predict_proba(x)
    a = np.max(pred_proba)
    z = pred_proba[0]*100
    
    # If Classification System
    if pred[0] == 'SP' or pred[0] == 'P':
        cv2.putText(img,'NYAMAN',(120,int(height-35)),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),3)
        cv2.putText(img,str(pred[0]),(100, int(height-75)),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
#         cv2.putText(img, str(z[0].round(2))+'%',(100, int(height-75)),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    else:
        cv2.putText(img,'TIDAK NYAMAN',(120,int(height-35)),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),3)
        cv2.putText(img,str(pred[0]),(100, int(height-75)),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)            
#         cv2.putText(img, str(z[0].round(2))+'%',(100, int(height-75)),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    
    # End of Prediction Time
    end = time.time()
    timeneeded = end - start
    
#     # Prediction Text
#     cv2.putText(img,'%Prediksi :',(10, int(height-80)), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)

    # satisfaction level
    cv2.putText(img,'Kepuasan :',(10, int(height-80)), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
    
    # Resp Name
    cv2.putText(img,'T(degree C) :', (10, int(height-140)), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
    cv2.putText(img,temperature,(120, int(height-140)), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
    
    cv2.putText(img,'Responden :', (10, int(height-110)), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
    cv2.putText(img,responden,(120, int(height-110)), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
    
    cv2.putText(img,'Kategori :',(10, int(height-50)), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
    cv2.putText(img,'Kenyamanan ',(10, int(height-30)), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
    
    datet=str(datetime.datetime.now())
    cv2.putText(img,datet,(10, 20),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
    cv2.putText(img,str(timeneeded)+'s',(10, 40),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
    cv2.imshow('Prediksi Kenyamanan Termal Personal- Realtime',img)
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

# sp_resp1_kondisix
# mp4
# Resp 2
# 25

Masukan nama file: allvideo
Masukan file format: mp4
Masukan nama responden: resp
Masukan nilai temperature ruangan: 25
